In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy import sparse

In [2]:
DATA_PATH = "./ppd_data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "cahr_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

SEED = 2018

In [3]:
train_raw = pd.read_csv(TRAIN_PATH)
test_raw = pd.read_csv(TEST_PATH)
question = pd.read_csv(QUEST_PATH)

In [4]:
question["words"] = question["words"].str.split(" ")
question["chars"] = question["chars"].str.split(" ")

In [5]:
train_total = pd.DataFrame(pd.concat([train_raw["q1"], train_raw["q2"]]), columns=["qid"])
train_total = train_total.merge(question, how="left", on="qid")
train_total.head()

,qid,words,chars
0,Q397345,"[W04465, W04058, W05284, W02916]","[L2218, L2568, L0360, L0242, L2218, L0741]"
1,Q193805,"[W10054, W04476, W09996, W12244, W18103]","[L2376, L2168, L0050, L1187, L0104, L2432, L09..."
2,Q085471,"[W04346, W17378, W19355, W17926, W14185, W1156...","[L2323, L1526, L2214, L1132, L2723, L1861, L22..."
3,Q189314,"[W17508, W09996, W19662, W17534, W11399, W1705...","[L0018, L2321, L1346, L2432, L0902, L1149, L19..."
4,Q267714,"[W13157, W03390, W01952, W05789, W17378, W0871...","[L2271, L1346, L1389, L2932, L0466, L2218, L19..."


In [6]:
train_words = train_total["words"].tolist()
train_chars = train_total["chars"].tolist()

In [7]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

word_dict = Dictionary(train_words)
char_dict = Dictionary(train_chars)
word_corpus = [word_dict.doc2bow(stc) for stc in train_words]
char_corpus = [char_dict.doc2bow(stc) for stc in train_chars]

word_tfidf_model = TfidfModel(word_corpus)
char_tfidf_model = TfidfModel(char_corpus)

In [8]:
train_word_q1, train_word_q2 = train_words[:len(train_raw)], train_words[len(train_raw):]
train_char_q1, train_char_q2 = train_chars[:len(train_raw)], train_chars[len(train_raw):]

In [9]:
word_corpus1 = [word_dict.doc2bow(stc) for stc in train_word_q1]
word_corpus2 = [word_dict.doc2bow(stc) for stc in train_word_q2]
char_corpus1 = [char_dict.doc2bow(stc) for stc in train_char_q1]
char_corpus2 = [char_dict.doc2bow(stc) for stc in train_char_q2]

In [10]:
word_tfidf1 = [word_tfidf_model[stc] for stc in word_corpus1]
word_tfidf2 = [word_tfidf_model[stc] for stc in word_corpus2]
char_tfidf1 = [char_tfidf_model[stc] for stc in char_corpus1]
char_tfidf2 = [char_tfidf_model[stc] for stc in char_corpus2]

In [11]:
def vec2sparse(vec, row_num, col_num):
    rows = [i for i, stc in enumerate(vec) for word in stc]
    cols = [word[0] for i, stc in enumerate(vec) for word in stc]
    data = [word[1] for i, stc in enumerate(vec) for word in stc]
    return sparse.csr_matrix((data, (rows, cols)), shape=(row_num, col_num))

In [12]:
word_sparse1 = vec2sparse(word_tfidf1, len(train_raw), len(word_dict.dfs))
word_sparse2 = vec2sparse(word_tfidf2, len(train_raw), len(word_dict.dfs))
char_sparse1 = vec2sparse(char_tfidf1, len(train_raw), len(char_dict.dfs))
char_sparse2 = vec2sparse(char_tfidf2, len(train_raw), len(char_dict.dfs))

In [13]:
from sklearn.model_selection import train_test_split

w1_train, w1_test, w2_train, w2_test, c1_train, c1_test, c2_train, c2_test, y_train, y_test = \
    train_test_split(word_sparse1, word_sparse2, char_sparse1, char_sparse2, train_raw["label"],
                     test_size=0.2, random_state=SEED, stratify=train_raw["label"])

### generate features by stacking

In [14]:
# X_stack_train = sparse.hstack([w1_train, c1_train, w2_train, c2_train])
# X_stack_test = sparse.hstack([w1_test, c1_test, w2_test, c2_test])
# X_stack_train, X_stack_test

# train_data = xgb.DMatrix(X_stack_train, label=y_train)
# dev_data = xgb.DMatrix(X_stack_test, label=y_test)
# train_data, dev_data

# params = {}
# params["objective"] = "binary:logistic"
# params["eval_metric"] = "logloss"
# params["seed"] = SEED
# params["eta"] = 0.01
# params["max_depth"] = 6

# watch_list = [(train_data, "train"), (dev_data, "dev")]
# boost = xgb.train(params, train_data, num_boost_round=50000, evals=watch_list, early_stopping_rounds=100, verbose_eval=10)

### generate features by sub

In [15]:
X_sub_train = sparse.hstack([w1_train - w2_train, c1_train - c2_train])
X_sub_test = sparse.hstack([w1_test - w2_test, c1_test - c2_test])
X_sub_train, X_sub_test

(<203508x13498 sparse matrix of type '<class 'numpy.float64'>'
 	with 5792961 stored elements in COOrdinate format>,
 <50878x13498 sparse matrix of type '<class 'numpy.float64'>'
 	with 1448513 stored elements in COOrdinate format>)

In [16]:
sub_train_data = xgb.DMatrix(X_sub_train, label=y_train)
sub_dev_data = xgb.DMatrix(X_sub_test, label=y_test)

sub_params = {}
sub_params["objective"] = "binary:logistic"
sub_params["eval_metric"] = "logloss"
sub_params["seed"] = SEED
sub_params["eta"] = 0.02
sub_params["max_depth"] = 5

sub_watch_list = [(sub_train_data, "train"), (sub_dev_data, "dev")]
sub_boost = xgb.train(sub_params, sub_train_data, num_boost_round=100000, evals=sub_watch_list,
                      early_stopping_rounds=100, verbose_eval=10)

[0]	train-logloss:0.690786	dev-logloss:0.690802
Multiple eval metrics have been passed: 'dev-logloss' will be used for early stopping.

Will train until dev-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.671249	dev-logloss:0.671509
[20]	train-logloss:0.655239	dev-logloss:0.655689
[30]	train-logloss:0.64251	dev-logloss:0.643119
[40]	train-logloss:0.631766	dev-logloss:0.632544
[50]	train-logloss:0.621635	dev-logloss:0.622587
[60]	train-logloss:0.612459	dev-logloss:0.61358
[70]	train-logloss:0.604683	dev-logloss:0.605881
[80]	train-logloss:0.597493	dev-logloss:0.598814
[90]	train-logloss:0.590834	dev-logloss:0.59233
[100]	train-logloss:0.58491	dev-logloss:0.586486
[110]	train-logloss:0.579317	dev-logloss:0.580983
[120]	train-logloss:0.574262	dev-logloss:0.576003
[130]	train-logloss:0.569337	dev-logloss:0.571155
[140]	train-logloss:0.564862	dev-logloss:0.566803
[150]	train-logloss:0.560625	dev-logloss:0.56263
[160]	train-logloss:0.5567	dev-logloss:0.558866
[170]	train-logloss:

[1610]	train-logloss:0.403998	dev-logloss:0.419235
[1620]	train-logloss:0.403611	dev-logloss:0.41892
[1630]	train-logloss:0.40314	dev-logloss:0.418515
[1640]	train-logloss:0.402818	dev-logloss:0.418259
[1650]	train-logloss:0.402424	dev-logloss:0.417911
[1660]	train-logloss:0.402082	dev-logloss:0.417627
[1670]	train-logloss:0.401632	dev-logloss:0.417248
[1680]	train-logloss:0.401283	dev-logloss:0.41698
[1690]	train-logloss:0.400915	dev-logloss:0.416666
[1700]	train-logloss:0.400503	dev-logloss:0.416328
[1710]	train-logloss:0.400196	dev-logloss:0.416084
[1720]	train-logloss:0.399794	dev-logloss:0.415756
[1730]	train-logloss:0.399457	dev-logloss:0.415495
[1740]	train-logloss:0.399094	dev-logloss:0.415202
[1750]	train-logloss:0.398723	dev-logloss:0.414894
[1760]	train-logloss:0.398352	dev-logloss:0.414608
[1770]	train-logloss:0.398017	dev-logloss:0.414339
[1780]	train-logloss:0.397697	dev-logloss:0.414073
[1790]	train-logloss:0.397362	dev-logloss:0.413825
[1800]	train-logloss:0.397008	dev-

[3230]	train-logloss:0.358935	dev-logloss:0.38423
[3240]	train-logloss:0.358703	dev-logloss:0.384042
[3250]	train-logloss:0.358457	dev-logloss:0.383874
[3260]	train-logloss:0.358231	dev-logloss:0.383696
[3270]	train-logloss:0.358002	dev-logloss:0.383532
[3280]	train-logloss:0.357828	dev-logloss:0.383414
[3290]	train-logloss:0.357653	dev-logloss:0.383289
[3300]	train-logloss:0.357383	dev-logloss:0.383097
[3310]	train-logloss:0.357204	dev-logloss:0.382976
[3320]	train-logloss:0.356989	dev-logloss:0.382804
[3330]	train-logloss:0.356826	dev-logloss:0.382694
[3340]	train-logloss:0.356606	dev-logloss:0.382525
[3350]	train-logloss:0.356398	dev-logloss:0.382364
[3360]	train-logloss:0.356171	dev-logloss:0.382203
[3370]	train-logloss:0.355968	dev-logloss:0.38204
[3380]	train-logloss:0.355781	dev-logloss:0.381906
[3390]	train-logloss:0.355609	dev-logloss:0.381794
[3400]	train-logloss:0.355411	dev-logloss:0.381665
[3410]	train-logloss:0.35521	dev-logloss:0.381528
[3420]	train-logloss:0.35502	dev-l

[4850]	train-logloss:0.332121	dev-logloss:0.365956
[4860]	train-logloss:0.331985	dev-logloss:0.36587
[4870]	train-logloss:0.331863	dev-logloss:0.365796
[4880]	train-logloss:0.331747	dev-logloss:0.365718
[4890]	train-logloss:0.331621	dev-logloss:0.365651
[4900]	train-logloss:0.331511	dev-logloss:0.365583
[4910]	train-logloss:0.331403	dev-logloss:0.365512
[4920]	train-logloss:0.331297	dev-logloss:0.365457
[4930]	train-logloss:0.331088	dev-logloss:0.365317
[4940]	train-logloss:0.330948	dev-logloss:0.365216
[4950]	train-logloss:0.330799	dev-logloss:0.365104
[4960]	train-logloss:0.330633	dev-logloss:0.364989
[4970]	train-logloss:0.330511	dev-logloss:0.364927
[4980]	train-logloss:0.33039	dev-logloss:0.364848
[4990]	train-logloss:0.330278	dev-logloss:0.364781
[5000]	train-logloss:0.33017	dev-logloss:0.364713
[5010]	train-logloss:0.330057	dev-logloss:0.364645
[5020]	train-logloss:0.329926	dev-logloss:0.36458
[5030]	train-logloss:0.329826	dev-logloss:0.364521
[5040]	train-logloss:0.329687	dev-l

[6470]	train-logloss:0.31303	dev-logloss:0.354249
[6480]	train-logloss:0.312927	dev-logloss:0.354202
[6490]	train-logloss:0.312802	dev-logloss:0.354125
[6500]	train-logloss:0.312722	dev-logloss:0.354094
[6510]	train-logloss:0.312634	dev-logloss:0.354041
[6520]	train-logloss:0.312545	dev-logloss:0.353982
[6530]	train-logloss:0.312441	dev-logloss:0.35392
[6540]	train-logloss:0.312312	dev-logloss:0.353835
[6550]	train-logloss:0.312212	dev-logloss:0.353774
[6560]	train-logloss:0.312118	dev-logloss:0.353713
[6570]	train-logloss:0.311991	dev-logloss:0.353648
[6580]	train-logloss:0.311891	dev-logloss:0.353588
[6590]	train-logloss:0.311807	dev-logloss:0.353538
[6600]	train-logloss:0.311663	dev-logloss:0.353446
[6610]	train-logloss:0.311551	dev-logloss:0.353384
[6620]	train-logloss:0.311466	dev-logloss:0.353331
[6630]	train-logloss:0.31134	dev-logloss:0.353249
[6640]	train-logloss:0.311264	dev-logloss:0.353213
[6650]	train-logloss:0.311135	dev-logloss:0.353143
[6660]	train-logloss:0.311069	dev-

...

[52640]	train-logloss:0.145936	dev-logloss:0.294755
[52650]	train-logloss:0.145906	dev-logloss:0.294749
[52660]	train-logloss:0.145885	dev-logloss:0.294746
[52670]	train-logloss:0.145866	dev-logloss:0.294745
[52680]	train-logloss:0.145841	dev-logloss:0.294746
[52690]	train-logloss:0.145822	dev-logloss:0.29475
[52700]	train-logloss:0.145805	dev-logloss:0.294749
[52710]	train-logloss:0.145789	dev-logloss:0.294742
[52720]	train-logloss:0.145767	dev-logloss:0.294739
[52730]	train-logloss:0.145742	dev-logloss:0.294735
[52740]	train-logloss:0.145722	dev-logloss:0.294733
[52750]	train-logloss:0.145697	dev-logloss:0.294727
[52760]	train-logloss:0.145683	dev-logloss:0.294726
[52770]	train-logloss:0.145653	dev-logloss:0.294721
[52780]	train-logloss:0.145641	dev-logloss:0.294717
[52790]	train-logloss:0.14563	dev-logloss:0.294717
[52800]	train-logloss:0.145599	dev-logloss:0.294711
[52810]	train-logloss:0.145583	dev-logloss:0.294706
[52820]	train-logloss:0.145572	dev-logloss:0.294705
[52830]	train-

[54230]	train-logloss:0.142593	dev-logloss:0.294184
[54240]	train-logloss:0.142579	dev-logloss:0.294178
[54250]	train-logloss:0.142566	dev-logloss:0.294179
[54260]	train-logloss:0.142534	dev-logloss:0.294166
[54270]	train-logloss:0.142518	dev-logloss:0.29417
[54280]	train-logloss:0.142498	dev-logloss:0.294167
[54290]	train-logloss:0.142471	dev-logloss:0.294157
[54300]	train-logloss:0.142457	dev-logloss:0.294158
[54310]	train-logloss:0.142441	dev-logloss:0.294153
[54320]	train-logloss:0.142422	dev-logloss:0.294151
[54330]	train-logloss:0.1424	dev-logloss:0.294149
[54340]	train-logloss:0.142385	dev-logloss:0.29415
[54350]	train-logloss:0.142372	dev-logloss:0.294147
[54360]	train-logloss:0.142358	dev-logloss:0.294147
[54370]	train-logloss:0.142334	dev-logloss:0.294148
[54380]	train-logloss:0.142313	dev-logloss:0.294147
[54390]	train-logloss:0.142298	dev-logloss:0.294146
[54400]	train-logloss:0.142273	dev-logloss:0.294136
[54410]	train-logloss:0.142243	dev-logloss:0.29412
[54420]	train-log

[55820]	train-logloss:0.139161	dev-logloss:0.293651
[55830]	train-logloss:0.139144	dev-logloss:0.293646
[55840]	train-logloss:0.139123	dev-logloss:0.293645
[55850]	train-logloss:0.139104	dev-logloss:0.293642
[55860]	train-logloss:0.139092	dev-logloss:0.293638
[55870]	train-logloss:0.139071	dev-logloss:0.293633
[55880]	train-logloss:0.139033	dev-logloss:0.293615
[55890]	train-logloss:0.139009	dev-logloss:0.293611
[55900]	train-logloss:0.138986	dev-logloss:0.293612
[55910]	train-logloss:0.138969	dev-logloss:0.293612
[55920]	train-logloss:0.138943	dev-logloss:0.293604
[55930]	train-logloss:0.138912	dev-logloss:0.2936
[55940]	train-logloss:0.138893	dev-logloss:0.293601
[55950]	train-logloss:0.138876	dev-logloss:0.2936
[55960]	train-logloss:0.138864	dev-logloss:0.293602
[55970]	train-logloss:0.138848	dev-logloss:0.293603
[55980]	train-logloss:0.138826	dev-logloss:0.293596
[55990]	train-logloss:0.138805	dev-logloss:0.293591
[56000]	train-logloss:0.138785	dev-logloss:0.29359
[56010]	train-log

[57410]	train-logloss:0.135836	dev-logloss:0.293135
[57420]	train-logloss:0.135818	dev-logloss:0.293134
[57430]	train-logloss:0.135808	dev-logloss:0.293136
[57440]	train-logloss:0.135789	dev-logloss:0.293135
[57450]	train-logloss:0.135766	dev-logloss:0.293131
[57460]	train-logloss:0.135739	dev-logloss:0.293124
[57470]	train-logloss:0.135716	dev-logloss:0.293123
[57480]	train-logloss:0.135685	dev-logloss:0.293114
[57490]	train-logloss:0.135674	dev-logloss:0.293112
[57500]	train-logloss:0.13566	dev-logloss:0.29311
[57510]	train-logloss:0.135637	dev-logloss:0.293104
[57520]	train-logloss:0.135618	dev-logloss:0.293099
[57530]	train-logloss:0.135599	dev-logloss:0.29309
[57540]	train-logloss:0.135574	dev-logloss:0.293085
[57550]	train-logloss:0.135551	dev-logloss:0.293081
[57560]	train-logloss:0.135542	dev-logloss:0.293078
[57570]	train-logloss:0.135531	dev-logloss:0.293079
[57580]	train-logloss:0.135514	dev-logloss:0.29308
[57590]	train-logloss:0.13549	dev-logloss:0.293073
[57600]	train-log

[59000]	train-logloss:0.132576	dev-logloss:0.292722
[59010]	train-logloss:0.132556	dev-logloss:0.292723
[59020]	train-logloss:0.132545	dev-logloss:0.292724
[59030]	train-logloss:0.132502	dev-logloss:0.292716
[59040]	train-logloss:0.13248	dev-logloss:0.292708
[59050]	train-logloss:0.132451	dev-logloss:0.292699
[59060]	train-logloss:0.132431	dev-logloss:0.2927
[59070]	train-logloss:0.132419	dev-logloss:0.292701
[59080]	train-logloss:0.132403	dev-logloss:0.292699
[59090]	train-logloss:0.132382	dev-logloss:0.292697
[59100]	train-logloss:0.132355	dev-logloss:0.292688
[59110]	train-logloss:0.132332	dev-logloss:0.292684
[59120]	train-logloss:0.132306	dev-logloss:0.292679
[59130]	train-logloss:0.132289	dev-logloss:0.292678
[59140]	train-logloss:0.132269	dev-logloss:0.292675
[59150]	train-logloss:0.132249	dev-logloss:0.292674
[59160]	train-logloss:0.132236	dev-logloss:0.292673
[59170]	train-logloss:0.132225	dev-logloss:0.292669
[59180]	train-logloss:0.13221	dev-logloss:0.292671
[59190]	train-lo

In [17]:
test_q1 = test_raw.merge(question, how="left", left_on="q1", right_on="qid").drop(["q1", "q2", "qid"], axis=1)
test_q2 = test_raw.merge(question, how="left", left_on="q2", right_on="qid").drop(["q1", "q2", "qid"], axis=1)

test_word1 = [word_tfidf_model[word_dict.doc2bow(stc)] for stc in test_q1["words"].tolist()]
test_word2 = [word_tfidf_model[word_dict.doc2bow(stc)] for stc in test_q2["words"].tolist()]
test_char1 = [char_tfidf_model[char_dict.doc2bow(stc)] for stc in test_q1["chars"].tolist()]
test_char2 = [char_tfidf_model[char_dict.doc2bow(stc)] for stc in test_q2["chars"].tolist()]

In [18]:
test_word_sparse1 = vec2sparse(test_word1, len(test_raw), len(word_dict.dfs))
test_word_sparse2 = vec2sparse(test_word2, len(test_raw), len(word_dict.dfs))
test_char_sparse1 = vec2sparse(test_char1, len(test_raw), len(char_dict.dfs))
test_char_sparse2 = vec2sparse(test_char2, len(test_raw), len(char_dict.dfs))

In [19]:
X_test = sparse.hstack([test_word_sparse1 - test_word_sparse2, test_char_sparse1 - test_char_sparse2])
X_test_data = xgb.DMatrix(X_test)
X_test_data

In [20]:
pred = sub_boost.predict(X_test_data)
pred

array([0.21560663, 0.9803411 , 0.09555773, ..., 0.919225  , 0.134799  ,
       0.0748089 ], dtype=float32)

In [21]:
pred_res = pd.Series(pred, name="y_pre")
pred_res.head()

0    0.215607
1    0.980341
2    0.095558
3    0.937629
4    0.542689
Name: y_pre, dtype: float32

In [22]:
pred_res.to_csv("./pred.csv", index=False, header=True)